In [3]:
# Mount Google Drive (to access dataset and save model)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
from google.colab import files

uploaded = files.upload()  # Upload .zip file

Saving Predict 5.mp4 to Predict 5.mp4


In [6]:
import zipfile

with zipfile.ZipFile('dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [7]:
import os
import cv2
import random

classes = ['Closed', 'Open', 'no_yawn', 'yawn']
base_path = '/content/dataset'
output_path = '/content/yolo_dataset'

for sub in ['images/train', 'images/val', 'labels/train', 'labels/val']:
    os.makedirs(os.path.join(output_path, sub), exist_ok=True)

def convert(x, y, w, h, iw, ih):
    return (x + w / 2) / iw, (y + h / 2) / ih, w / iw, h / ih

for phase in ['train', 'test']:
    for cls_id, cls_name in enumerate(classes):
        img_dir = os.path.join(base_path, phase, cls_name)
        img_files = os.listdir(img_dir)
        for img_file in img_files:
            img_path = os.path.join(img_dir, img_file)
            img = cv2.imread(img_path)
            if img is None: continue
            h, w = img.shape[:2]
            x_center, y_center, bw, bh = convert(0, 0, w, h, w, h)

            # Rename path
            name = os.path.splitext(img_file)[0] + '.jpg'
            target_folder = 'train' if phase == 'train' else 'val'
            out_img_path = os.path.join(output_path, 'images', target_folder, name)
            out_lbl_path = os.path.join(output_path, 'labels', target_folder, name.replace('.jpg', '.txt'))

            cv2.imwrite(out_img_path, cv2.resize(img, (320, 320)))
            with open(out_lbl_path, 'w') as f:
                f.write(f"{cls_id} {x_center} {y_center} {bw} {bh}\n")

print("✅ Dataset converted to YOLO format.")

✅ Dataset converted to YOLO format.


In [8]:
# Define YAML content
data_yaml = """
path: /content/yolo_dataset
train: images/train
val: images/val

names:
  0: Closed
  1: Open
  2: no_yawn
  3: yawn
"""

# Save it as data.yaml in the dataset directory
with open('/content/yolo_dataset/data.yaml', 'w') as f:
    f.write(data_yaml)

print("✅ data.yaml created at /content/yolo_dataset/data.yaml")

✅ data.yaml created at /content/yolo_dataset/data.yaml


In [9]:
!pip install ultralytics --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.5/983.5 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [2]:
from ultralytics import YOLO
import torch

print("Ultralytics YOLO version:", YOLO)
print("✅ GPU is available!" if torch.cuda.is_available() else "❌ GPU not detected")

Ultralytics YOLO version: <class 'ultralytics.models.yolo.model.YOLO'>
✅ GPU is available!


In [3]:
from ultralytics import YOLO

# Load a small pre-trained model (yolov8n is fastest, yolov8s is balanced)
model = YOLO('yolov8s.pt')  # use 'yolov8s.pt' if you want more accuracy

model.train(
    data='/content/yolo_dataset/data.yaml',
    epochs=25,             # reduced from 500
    imgsz=416,             # keep for decent accuracy
    batch=4,               # reduced to prevent OOM crashes
    device=0,              # GPU
    name='drowsy_detector',
    save_period=5,         # checkpoint every 5 epochs
    patience=10,           # early stop if no val improvement
    resume=False,          # don't resume unless you've already trained
    workers=2              # reduce worker threads (Colab can crash if >2)
)

Ultralytics 8.3.115 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/yolo_dataset/data.yaml, epochs=25, time=None, patience=10, batch=4, imgsz=416, save=True, save_period=5, cache=False, device=0, workers=2, project=None, name=drowsy_detector2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, l

train: Scanning /content/yolo_dataset/labels/train.cache... 1699 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1699/1699 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 598.7±387.3 MB/s, size: 25.2 KB)


val: Scanning /content/yolo_dataset/labels/val.cache... 318 images, 0 backgrounds, 0 corrupt: 100%|██████████| 318/318 [00:00<?, ?it/s]


Plotting labels to runs/detect/drowsy_detector2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 416 train, 416 val
Using 2 dataloader workers
Logging results to runs/detect/drowsy_detector2
Starting training for 25 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        318        318      0.996      0.997      0.994      0.994
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 14, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



24 epochs completed in 0.293 hours.
Optimizer stripped from runs/detect/drowsy_detector2/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/drowsy_detector2/weights/best.pt, 22.5MB

Validating runs/detect/drowsy_detector2/weights/best.pt...
Ultralytics 8.3.115 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,127,132 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 40/40 [00:03<00:00, 10.99it/s]


                   all        318        318      0.992      0.994      0.995      0.995
                  Open        109        109      0.995          1      0.995      0.995
               no_yawn        103        103       0.99       0.99      0.995      0.995
                  yawn        106        106      0.992      0.991      0.995      0.995
Speed: 0.2ms preprocess, 3.3ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to runs/detect/drowsy_detector2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d45a254b690>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

In [4]:
# Load the trained model
model = YOLO("runs/detect/drowsy_detector/weights/best.pt")

# Evaluate on validation set
metrics = model.val()

# This will return a dictionary with metrics
print(metrics.box.map)       # mAP@0.5

Ultralytics 8.3.115 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,127,132 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1354.6±527.3 MB/s, size: 30.6 KB)


val: Scanning /content/yolo_dataset/labels/val.cache... 318 images, 0 backgrounds, 0 corrupt: 100%|██████████| 318/318 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:03<00:00,  5.76it/s]


                   all        318        318      0.992      0.994      0.995      0.995
                  Open        109        109      0.995          1      0.995      0.995
               no_yawn        103        103       0.99       0.99      0.995      0.995
                  yawn        106        106      0.992      0.991      0.995      0.995
Speed: 0.3ms preprocess, 4.7ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/val9
0.9950000000000002
